## Custom written code to generate Figure 3 and Figure 4 for Liu 2023

Figure 3. sum projections of group averages

Figure 4. serial coronal sections

Also prepares intermediate files (xlsx) for the subsequent figures

In [2]:
import os

import pandas as pd

import numpy as np

import SimpleITK as sitk

import warnings

import tkinter.filedialog as fdialog

import random

import matplotlib.pyplot as plt
import Neuron_analysis as na
from Neuron_analysis import *
import skimage
from skimage import io

import re
from tqdm import tqdm

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

ModuleNotFoundError: No module named 'Neuron_analysis'

### Figure 3

Sum projections of group averages in coronal, horizontal and saggital views


In [ ]:
region='s1'
# or 's2'

indir = f'D:\\viral_results\\{region}\\npy'
#specifiy input folder containing the horizontal npy files

outdir =f'D:\\viral_results\\{region}_average'
# define folder to store the results

in_files= os.listdir(indir)

sample_files= os.listdir(indir)



In [ ]:
def save_tiff(numpystack, outname):
    
    '''save numpy stack as tiff and sum max projection in horizontal, coronal and saggital views'''
    
    print('Starting to saving tif files..')
    
    io.imsave(outname+'_axons.tif',numpystack,check_contrast=False)
    
    sum_0=np.sum(numpystack, axis=0)
    sum_1=np.sum(numpystack, axis=1)
    sum_2=np.sum(numpystack, axis=2)
    io.imsave(outname+ '_sum_0.tif',sum_0,check_contrast=False)
    io.imsave(outname+ '_sum_1.tif',sum_1,check_contrast=False)
    io.imsave(outname+ '_sum_2.tif',sum_2,check_contrast=False)

In [ ]:
# DICTIONARY FOR SAMPLES!!

if region == 's1':
    sample_dictionary= {'sim': ['AL207','AL209','AL273'],
                        'rbp': ['AL211','AL318'],
                        'tlx': ['AL213','AL313','AL314'],
                        'ras': ['AL254','AL255','AL257'],
                        'scn': ['AL290','AL291','AL292','AL293'],
                        'nts': ['AL274','AL285','AL311']
                       }
elif region == 's2':
    sample_dictionary= {'sim': ['AL281','AL286','AL321','AL322'],
                        'rbp': ['AL288','AL326','AL327'],
                        'tlx': ['AL278','AL280','AL319'],
                        'ras': ['AL303','AL332','AL333'],
                        'scn': ['AL290','AL292','AL323'],
                        'nts': ['AL274','AL310','AL330']
                       }

In [ ]:
for genotype, sample_name in tqdm(sample_dictionary.items()):
    
    print ('Working on '+ genotype + ' samples') 
    
    out_name= outdir+ f'\\{genotype}_average'
    
    relevant_npy_name= [i+'.npy' for i in sample_name]
    # get the relevant names
    
    template=np.zeros([320, 528, 456])
    # initialize np array
    
    for i in relevant_npy_name:
        template+= np.load(os.path.join(indir,i))
    
    averaged_stack= template/ len(relevant_npy_name)
    # average across number of involved samples
    
    np.save(out_name+'.npy', averaged_stack)
    
    averaged_stack[0,0,0]=0
    averaged_stack[1,0,0]=0
    # reassign these pixels back to 0
    
    save_tiff(averaged_stack, out_name)

### Figure 4 WORK IN PROGRESS

Serial coronal sections of group averages

Sum projections of a few coronal sections centered around a given AP location
- ie. 125 um thickness centered around AP +3 mm to -7 mm in 1 mm intervals

In [ ]:
outdir= r'D:\viral_results\consecutive_coronal'

In [21]:
regions=['s1', 's2']
# define the two injection regions

mouse_lines=['ras','scn','tlx','sim','rbp','nts']
# defined the list of transgenic lines

bregma= na.bregma[2]
# bregma is section 215 in a coronal stack in the Allen brain 25um CCF

NameError: name 'na' is not defined

In [22]:
start = -3
# starting at 3 AP, note the negative sign

end = 7
# ending at -7 AP, note the positive sign

thickness= 5
pre= np.floor(thickness/2)
post=np.floor(1+thickness/2)

In [23]:
# loop through group averages

for i in regions:
    
    
    for j in tqdm(mouse_lines):    
        
        file_path= f'D:\\viral_results\\{i}_average\\{j}_average.npy'

        axons=np.load(file_path)
        # load horizontal npy file of group averages
        
        axons[0,0,0]=0
        axons[1,0,0]=0
        # restore these values back to 0
        
        coronal= axons.swapaxes(0,1)
        # reshape axons into coronal sections
        
        for k in range(start,end-1):
            
            #take -3 mm to  7 mm in 1 mm interval points with 5 sections (125 um) around this point, in terms of stereotaxic cordinate where bregma= 0
            
            midpoint= int( bregma + k/0.025)
            # convert this stereotaxic point back where 1 pixel= 25 microns or 0.025 mm
            
            this_plane=coronal[midpoint-pre : midpoint+post,:,:].sum(0)
            # keep 2 slices before and 2 slices after the midpoint, which is 5 slices in total= 125um thickness

            out_name= f'{outdir}\\{i}_{j}_avg_AP{k}'

            #io.imsave(out_name+ '_sum.tif',this_plane,check_contrast=False)

NameError: name 'tqdm' is not defined

In [24]:
# cleaned up code above!

# original code here


outdir= r'D:\viral_results\consecutive_coronal'



for i in regions:
    for j in tqdm(mouse_lines):
        
        
        file_path= f'D:\\viral_results\\{i}_average\\{j}_average.npy'

        axons=np.load(file_path)
        axons[0,0,0]=0
        axons[1,0,0]=0
        # restore these values back to 0
        
        coronal= axons.swapaxes(0,1)
        # reshape axons into coronal sections
        
        for k in range(stereo_range):
            #take -3 mm to 8 mm in 1 mm interval points with 5 sections (125 um) around this point, in terms of stereotaxic cordinate where bregma= 0
            
            midpoint= int( bregma + k/0.025)
            # convert this stereotaxic point back
            
            this_plane=coronal[midpoint-2 : midpoint+3,:,:].sum(0)
            # keep 2 slices before and 2 slices after the midpoint, which is 5 slices in total= 125um thickness

            out_name= f'{outdir}\\{i}_{j}_avg_AP{k}'

            io.imsave(out_name+ '_sum.tif',this_plane,check_contrast=False)
        

NameError: name 'tqdm' is not defined

### Preparation categorical data as excel files for Figures 5,6 and 7
Numbers of of axons in brain region X

Should get 4 xlsx files per sample

In [26]:
# useful functions

def find_points_id_from_npy(npy_file_path):
    
    ''' quantify amount of axons in brain region from npy files
    '''
    
    sample_array=np.load(npy_file_path)
    
    sample_array[0,0,0]=0
    sample_array[1,0,0]=0
    # reassign position 0,0,0 and 1,0,0 as 0
    # this is because we stored number of axons (at 0,0,0) and number of cells at(1,0,0) at these locations 

    sample_h=np.swapaxes(sample_array, 0, 2)
    # swap axis to match the atlas shape, this is the axon in horizontal view
    
    axons= np.argwhere(sample_h)
    # find all the non-zero indices from the stack
    
    return sample_h, axons

def get_subregion_excel(axon_points):
    
    ''' Quantify axon counts in the most detailed anatomical region possible
    '''
    
    region_in_atlas= []
    for i in axon_points:
        region_in_atlas.append(int(annot_h[i[0], i[1],i[2]]))
    #get atlas id for each non-zero indices on this side side

    axon_values=[]
    for i in axon_points:
        axon_values.append (sample_h[i[0],i[1],i[2]])
    # get the actual count for each non-zero indices on this side

    this_data=pd.DataFrame({'id': region_in_atlas, 'counts': axon_values})
    # store in data frame

    this_data=this_data.groupby(by='id').sum()
    # add the values from a given brain region together
    
    this_df=pd.merge(atlas_labels, this_data, on='id')
    # add in additional information from allen labels and save
    
    return this_df

def parent_df(df):
    ''' 
    Used under the get_parent_excel function
    group dataframe by parent id structure'''
    
    grouped_pd=df.groupby(['parent_structure_id'],as_index=False).sum()
    d= {'id': grouped_pd.parent_structure_id.astype(int), 'counts': grouped_pd.counts}
    grouped_pd2= pd.DataFrame(data=d)
    result = pd.merge(grouped_pd2, na.atlas_labels, on=["id"])
    result.sort_values(['counts'], ascending=True, inplace=True)
    # result is the final pd

    return result

def clean_duplicate(df):
    '''
    Used under the get_parent_excel function
    This is needed again to account for parent regions that have its subregion incompletely covers the parent areas.
    A painful example is Zona incerta, since it has a depth of 6, after group by parents it will show up twice with different counts where we simply just add the two counts
    '''
    
    df2=df.groupby(['acronym'],as_index=False, sort=False).sum()
    d= {'acronym': df2.acronym, 'counts': df2.counts}
    grouped_pd2= pd.DataFrame(data=d)
    result = pd.merge(grouped_pd2, na.atlas_labels, on=["acronym"])
    # this merging is required because pd.groupby will drop 'useless columns' such as depth, structure id path and other useful ones!! So we fetch them back here..
    # Probably have better ways of doing it...
    
    return result
def get_parent_excel(subregion_df):
    ''' quantify axon counts in broader anatomical regions, where regions with depth>6 will be grouped one level up
    this is done so that layers of cerebral cortex gets grouped together (ie, ssp-bfd l2/3,l4, l5 and l6 becomes ssp-bfd)
    '''

    axon_sub=subregion_df.sort_values(by=['counts'])
    axon_sub.sort_values(by= 'graph_order',axis=0, inplace=True)

    needsgroup=axon_sub[axon_sub.depth>6]
    noneedsgroup=axon_sub[axon_sub.depth<=6]

    parent= parent_df(needsgroup)

    complete1=noneedsgroup.append(parent)

    complete=clean_duplicate(complete1)

    complete.sort_values('counts',ascending=False, inplace=True)


In [25]:
region='s1'
# or 's2'

indir = f'D:\\viral_results\\{region}\\npy'
#specifiy input folder containing the horizontal npy files


in_files= os.listdir(indir)

sample_files= os.listdir(indir)

outdir =f'D:\\viral_results\\{region}_result'
# define folder to store the results


In [ ]:
mcc = MouseConnectivityCache(resolution=25)
annot, annot_info = mcc.get_annotation_volume()
print('Coronal atlas has shape', annot.shape)
# load allen mouse brain atlas that is in coronal view

annot_h=np.moveaxis(annot, 2, 0)
print('Converted to horizontal atlas with shape', annot_h.shape)
# reslice corontal atlas to horizontal atlas that is consistent with our sample's orientation

atlas_labels=pd.read_csv('D:\Allenbrainatlas\ARA_25_micron_mhd_ccf2017\labels.csv')
# read axon labels

In [ ]:
for i in tqdm(sample_files):
    
    this_file= os.path.join(indir,i)
    
    mouse_name= na.find_mousename(this_file)
    # identify mouse id

    out_name= os.path.join(outdir,mouse_name)
    #define a generic output file name for this sample

    #print(f'now processing {mouse_name}')
    
    sample_h,this_axon= find_points_id_from_npy(this_file)
    # read sample and axon indices

    left_points= []
    right_points= []
    for item in this_axon:
        if item[0]<227:
            left_points.append(item)
        else:
            right_points.append(item)
    # separate in to left and right hemisphere, defined by splitting the horizontal image down the middle

    left_df= get_subregion_excel(left_points)
    left_df.to_excel(out_name+'_left_region_with_counts.xlsx',index=None,header=True)

    right_df= get_subregion_excel(right_points)
    right_df.to_excel(out_name+'_right_region_with_counts.xlsx',index=None,header=True)


    left_parent= get_parent_excel(left_df)
    left_parent.to_excel(f'{out_name}_Lparent.xlsx') 
    #print('left_parent excel saved')

    if len(right_points)==0:
        right_df.to_excel(f'{out_name}_Rparent.xlsx') 
    else:
        right_parent= get_parent_excel(right_df)
        right_parent.to_excel(f'{out_name}_Rparent.xlsx') 
    #print('right_parent excel saved')
    